In [1]:
# ========= Importation des libraries ======= #
import dash
from dash import html, dcc, dash_table
import plotly.graph_objects as go
import dash_bootstrap_components as dbc 
import pandas as pd
import numpy as np
from calendar import month_abbr, month_name
import plotly.express as px
from dash.dependencies import Input, Output

# ========= DATA ======= #

df = pd.read_csv("./data.csv", index_col=0)
df['Total_price'] = df['Quantity'] * df['Avg_Price'] * (1 - df['Discount_pct']/100)
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])

Locations = df["Location"].dropna().unique() #Variable locations contenant les valeurs uniques de la colonne Location

# ========= Fonctions / graphs ======= #

def calculer_chiffre_affaire(data):
    return data['Total_price'].sum()

def frequence_meilleure_vente(data, top=10, ascending=False):
    resultat = pd.crosstab(
        [data['Gender'], data['Product_Category']], 
        'Total vente', 
        values=data['Total_price'], 
        aggfunc= lambda x : len(x), 
        rownames=['Sexe', 'Categorie du produit'],
        colnames=['']
    ).reset_index().groupby(
        ['Sexe'], as_index=False, group_keys=True
    ).apply(
        lambda x: x.sort_values('Total vente', ascending=ascending).iloc[:top, :]
    ).reset_index(drop=True).set_index(['Sexe', 'Categorie du produit'])

    return resultat

def indicateur_du_mois(data, current_month = 12, freq=True, abbr=False): 
    previous_month = current_month - 1 if current_month > 1 else 12
    if freq : 
        resultat = data['Month'][(data['Month'] == current_month) | (data['Month'] == previous_month)].value_counts()
        resultat = resultat.sort_index()
        resultat.index = [(month_abbr[i] if abbr else month_name[i]) for i in resultat.index]
        return resultat
    else:
        resultat = data[(data['Month'] == current_month) | (data['Month'] == previous_month)].groupby('Month').apply(calculer_chiffre_affaire)
        resultat.index = [(month_abbr[i] if abbr else month_name[i]) for i in resultat.index]
        return resultat

def barplot_top_10_ventes(data):
    df_plot = frequence_meilleure_vente(data, ascending=True)
    graph = px.bar(
        df_plot.reset_index(),
        x='Total vente', 
        y='Categorie du produit',
        color='Sexe', 
        barmode='group',
        title="Frequence des 10 meilleures ventes",
        labels={"Total vente": "Fréquence", "Categorie du produit": "Categorie du produit", "Sexe": "Sexe"},
        width=700, height=650
    ).update_layout(
        margin = dict(t=60)
    )
    return graph

def plot_evolution_chiffre_affaire(data):
    df_plot = data.groupby(pd.Grouper(key='Transaction_Date', freq='W')).apply(calculer_chiffre_affaire)[:-1]
    chiffre_evolution = px.line(
        x=df_plot.index, y=df_plot,
        title="Evolution du chiffre d'affaire par semaine",
        labels={"x": "Semaine", "y": "Chiffre d'affaire"},
    ).update_layout( 
        width=1000, height=400,
        margin=dict(t=60, b=0),
    )
    return chiffre_evolution

def plot_chiffre_affaire_mois(data) :
    df_plot = indicateur_du_mois(data, freq=False)
    indicateur = go.Figure(
        go.Indicator(
            mode = "number+delta",
            value = df_plot[1],
            delta = {'reference': df_plot[0]},
            domain = {'row': 0, 'column': 1},
            title=f"{df_plot.index[1]}",
        )
    ).update_layout(
        width=200, height=200, 
        margin=dict(l=0, r=20, t=20, b=0)
    )
    return indicateur

def plot_vente_mois(data, abbr=False) :
    df_plot = indicateur_du_mois(data, freq=True, abbr=abbr)
    indicateur = go.Figure(
        go.Indicator(
            mode = "number+delta",
            value = df_plot[1],
            delta = {'reference': df_plot[0]},
            domain = {'row': 0, 'column': 1},
            title=f"{df_plot.index[1]}",
        )
    ).update_layout( 
        width=200, height=200, 
        margin=dict(l=0, r=20, t=20, b=0)
    )
    return indicateur

# Table des 100 dernières ventes
def table_ventes(data):
    df_copy = data.copy()
    df_copy['Transaction_Date'] = df_copy['Transaction_Date'].dt.date
    df_copy = df_copy.sort_values('Transaction_Date', ascending=False).head(100)
    
    columns = [
        {"name": "Date", "id": "Transaction_Date"},
        {"name": "Genre", "id": "Gender"},
        {"name": "Location", "id": "Location"},
        {"name": "Catégorie produit", "id": "Product_Category"},
        {"name": "Quantité", "id": "Quantity"},
        {"name": "Prix moyen", "id": "Avg_Price"},
        {"name": "Réduction (%)", "id": "Discount_pct"}
    ]
    
    return {
        'data': df_copy.to_dict('records'),
        'columns': columns
    }   


# ======= Initialisation de l'application ======= #
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# ============ Layout =============== #
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1('ECAP Store'), width=6),
        dbc.Col(
            dcc.Dropdown(
                id='location-liste',
                options=[
                    {'label': 'Toutes les localisations', 'value': 'Toutes_les_locs'}, #Rajout de l'option Toutes les localisations dans le dropdown 
                    *[{'label': loc, 'value': loc} for loc in Locations]
                ],
                value='Toutes_les_locs', 
                style={'width': '70%', 'margin': '0 auto'},
                placeholder="Sélectionnez une localisation"  
            ),className="d-flex align-items-center justify-content-center"
            
        ),
    ], style={"background-color": "lightblue", "color": "black", "padding": "10px"}),
    dbc.Row([
        dbc.Col([
            dbc.Row([
                dbc.Col(dcc.Graph(id="monthly-revenue", figure=plot_chiffre_affaire_mois(df)), width=6, className="d-flex justify-content-center", style={"margin-top": "30px"}),
                dbc.Col(dcc.Graph(id="monthly-sales", figure=plot_vente_mois(df)), width=6, className="d-flex justify-content-center", style={"margin-top": "30px"})
            ]),
           dbc.Row([
                dbc.Col(dcc.Graph(id="top-10-sales", figure=barplot_top_10_ventes(df))),
            ]),
        ], width=5),
        dbc.Col([
            dbc.Row([
                dcc.Graph(id="revenue-evolution", figure=plot_evolution_chiffre_affaire(df), style={"margin-top": "30px"}) 
            ]),

            dbc.Row(children=[
                dbc.Col([
                html.H5(children='Table des 100 dernières ventes', style={'marginRight': '30px'}),
                dash_table.DataTable(
                                    id='Table', 
                                    style_table={'width': '100%', "height": "50%"},
                                    editable=False,
                                    filter_action='native',
                                    sort_action='native',
                                    page_action='native',
                                    page_current=0,
                                    page_size=10,
                                    data=table_ventes(df)['data'],
                                    columns=table_ventes(df)['columns']
                                    ),
                ]),
            ]),
        ]),
    ]),
], fluid=True)

# Callback pour mettre à jour le graphique de ventes par localisation
@app.callback(
    [Output("monthly-sales", 'figure'),
     Output("monthly-revenue", 'figure'),
     Output("top-10-sales", 'figure'),
     Output("revenue-evolution", 'figure'),
     Output('Table', 'data')],
    [Input('location-liste', 'value')]
)
def update_graphs(location_selec):    
    if location_selec is None or location_selec == 'Toutes_les_locs':
        data_filtre = df
    else:
        data_filtre = df[df["Location"] == location_selec]
    
    monthly_sales_fig = plot_vente_mois(data_filtre)
    monthly_revenue_fig = plot_chiffre_affaire_mois(data_filtre)
    top_10_sales_fig = barplot_top_10_ventes(data_filtre)
    revenue_evolution_fig = plot_evolution_chiffre_affaire(data_filtre)
    table_100_ventes = table_ventes(data_filtre)['data']
    
    return monthly_sales_fig, monthly_revenue_fig, top_10_sales_fig, revenue_evolution_fig, table_100_ventes

# ================ Run server ================== #
if __name__ == '__main__':
    app.run_server(debug=True, port=8051, jupyter_mode = "External")

FileNotFoundError: [Errno 2] No such file or directory: './data.csv'